In [49]:
from itertools import chain, combinations
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd
import calendar
import time
from sklearn.metrics import accuracy_score, precision_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [50]:
# Train the model
def trainModel(X_train, y_train):
    print(X_train)
    # Set regularization rate
    reg = 0.01
    # train a logistic regression model on the training set
    mod = LogisticRegression(C=1/reg, solver="liblinear", random_state=42).fit(X_train, y_train)

    return mod

def custom_predict(X, threshold, model):
    probs = model.predict_proba(X) 
    return (probs[:, 1] > threshold).astype(int)

In [51]:
dataframe = pd.DataFrame(columns=['DataSetName', 'TrainAccuracy', 'TrainPrecision', 'TrainF2', 'TrainRecall',
                                  'TrainROC', 'TestAccuracy', 'TestPrecision', 'TestF2', 'TestRecall', 'TestROC', 'BestParams'])

features = ['HomeGoalsCumulative',	'AwayGoalsCumulative',	'HomePointsCumulative',
            'AwayPointsCumulative',	'HomeGoalsConcededCumulative',	'AwayGoalsConcededCumulative']

combinazioni = chain.from_iterable(combinations(
    features, r) for r in range(1, len(features) + 1))

combos = []

for idx, combinazione in enumerate(combinazioni):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    print(idx, time.strftime("%H:%M:%S", time.localtime()), ':', combinazione)
    combos.append(combinazione)

print(combos)

for threshold in range(5, 80, 5):
    threshold /= 100
    data = pd.read_excel('data/mergedData.xlsx')
    data['Cluster'] = np.where(data['FTHG'] == data['FTAG'], 1, 0)

    # data['Cluster'] = np.where((data['Close'] < data['Open']) & (
    #     abs(data['Close'] - data['Open'])/data['Open'] > bodyRange/10000), -1, 1)

    # Select 'feat' columns

    print("Extracting features columns and creating target variable...")

    for idx, combo in enumerate(combos):
        current_GMT = time.gmtime()
        time_stamp = calendar.timegm(current_GMT)
        print(idx, time.strftime("%H:%M:%S", time.localtime()),
                'Trying with following combo', combo)
        feat_cols = [col for col in combo]

        x_train, x_test, y_train, y_test = train_test_split(
            data[feat_cols], data['Cluster'], test_size=0.3, random_state=42, shuffle=True)

        x_train = x_train.sort_index()
        x_test = x_test.sort_index()
        y_train = y_train.sort_index()
        y_test = y_test.sort_index()


        model = trainModel(x_train, y_train)

        # Predictions
        y_train_pred = model.predict(x_train.sort_index())
        y_test_pred = model.predict(x_test.sort_index())
        text_pred_prob = model.predict_proba(x_test)
        # print('y_test_pred', y_test_pred)
        # clf.fit(x_train,y_train)

        # y_train_pred = clf.predict(x_train)
        # y_test_pred = clf.predict(x_test)

        # VN Calcolo la equity

        test_data = data[data.index.isin(x_test.index)]
        # test_data['prediction'] = y_test_pred
        test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
        # print('test_data[predictions]', test_data['predictions'])
        test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
                                        test_data['B365D']-1, np.where((test_data['FTR'] != 'D') & (test_data['predictions'] > threshold), -1, 0))
        test_equity = test_data['gain'].sum()
        # print('test_data',test_data)

        print("Train metrics...")
        # Metrics
        # train_accuracy = accuracy_score(y_train, y_train_pred)
        # train_precision = precision_score(y_train, y_train_pred)

        print("Test metrics...")
        test_accuracy = accuracy_score(y_test, test_data['predictions'])
        # test_precision = precision_score(y_test, y_test_pred)
        # test_f1 = f1_score(y_test, y_test_pred)

        # Save Results
        new_row = pd.Series({
            'DataSetName': str(combo) + str(threshold),
            # 'TrainAccuracy': train_accuracy,
            # 'TrainPrecision': train_precision,
            'TestAccuracy': test_accuracy,
            # 'TestPrecision': test_precision,
            # 'TestF1': test_f1,
            'BestParams': model.get_params,
            'TestEquity': test_equity,
            # 'TestCagr': test_cagr,
            # 'TestMaxDD': test_max_dd,
            # 'Eseguiti': numOp
        })
        print(new_row)
        dataframe = pd.concat(
            [dataframe, new_row.to_frame().T], ignore_index=True)

dataframe.to_excel("Dataframe/"+str(time_stamp) + ".xlsx")

# print(result)

0 22:42:40 : ('HomeGoalsCumulative',)
1 22:42:40 : ('AwayGoalsCumulative',)
2 22:42:40 : ('HomePointsCumulative',)
3 22:42:40 : ('AwayPointsCumulative',)
4 22:42:40 : ('HomeGoalsConcededCumulative',)
5 22:42:40 : ('AwayGoalsConcededCumulative',)
6 22:42:40 : ('HomeGoalsCumulative', 'AwayGoalsCumulative')
7 22:42:40 : ('HomeGoalsCumulative', 'HomePointsCumulative')
8 22:42:40 : ('HomeGoalsCumulative', 'AwayPointsCumulative')
9 22:42:40 : ('HomeGoalsCumulative', 'HomeGoalsConcededCumulative')
10 22:42:40 : ('HomeGoalsCumulative', 'AwayGoalsConcededCumulative')
11 22:42:40 : ('AwayGoalsCumulative', 'HomePointsCumulative')
12 22:42:40 : ('AwayGoalsCumulative', 'AwayPointsCumulative')
13 22:42:40 : ('AwayGoalsCumulative', 'HomeGoalsConcededCumulative')
14 22:42:40 : ('AwayGoalsCumulative', 'AwayGoalsConcededCumulative')
15 22:42:40 : ('HomePointsCumulative', 'AwayPointsCumulative')
16 22:42:40 : ('HomePointsCumulative', 'HomeGoalsConcededCumulative')
17 22:42:40 : ('HomePointsCumulative', '

C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is tryin

Train metrics...
Test metrics...
DataSetName     ('AwayGoalsCumulative', 'AwayGoalsConcededCumu...
TestAccuracy                                             0.263158
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                   8.49
dtype: object
15 22:42:43 Trying with following combo ('HomePointsCumulative', 'AwayPointsCumulative')
      HomePointsCumulative  AwayPointsCumulative
0                        0                     0
1                        0                     0
2                        0                     0
3                        0                     0
4                        0                     0
...                    ...                   ...
1895                    33                    63
1896                    35                    34
1897                    60                    29
1898                    37                    93
1899                    69                    57

[1330 rows 

C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is tryin

      HomeGoalsCumulative  HomePointsCumulative  AwayPointsCumulative
0                       0                     0                     0
1                       0                     0                     0
2                       0                     0                     0
3                       0                     0                     0
4                       0                     0                     0
...                   ...                   ...                   ...
1895                   27                    33                    63
1896                   44                    35                    34
1897                   48                    60                    29
1898                   36                    37                    93
1899                   70                    69                    57

[1330 rows x 3 columns]
Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'HomePointsCumulative'...
TestAccuracy                        

...

C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is tryin

Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'AwayPointsCumulative'...
TestAccuracy                                             0.736842
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                    0.0
dtype: object
51 22:43:46 Trying with following combo ('AwayGoalsCumulative', 'HomePointsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative')
      AwayGoalsCumulative  HomePointsCumulative  AwayPointsCumulative  \
0                       0                     0                     0   
1                       0                     0                     0   
2                       0                     0                     0   
3                       0                     0                     0   
4                       0                     0                     0   
...                   ...                   ...                   ...   
1895                   64          

C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is tryin

Train metrics...
Test metrics...
DataSetName     ('AwayGoalsCumulative', 'HomePointsCumulative'...
TestAccuracy                                             0.736842
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                    0.0
dtype: object
62 22:43:46 Trying with following combo ('HomeGoalsCumulative', 'AwayGoalsCumulative', 'HomePointsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative')
      HomeGoalsCumulative  AwayGoalsCumulative  HomePointsCumulative  \
0                       0                    0                     0   
1                       0                    0                     0   
2                       0                    0                     0   
3                       0                    0                     0   
4                       0                    0                     0   
...                   ...                  ...              

C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_6052\3281331788.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
